In [1]:
import pandas as pd
from datetime import datetime
import requests
import time 
from requests_html import HTMLSession
# from requests.packages.urllib3.exceptions import InsecureRequestWarning
from PIL import Image
from io import StringIO, BytesIO

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from IPython.display import display, clear_output


In [3]:
! pip install --upgrade selenium

In [2]:
# s = Service('chromedriver')
# driver = webdriver.Chrome(service=s)
session = HTMLSession()
start = time.time()
pages = 100
single_page_urls = []
page_cnt, url_cnt = 0, 0
for p in range(1, pages+1):
    bikroy_url = f'https://bikroy.com/en/ads/bangladesh/home-living?sort=date&order=desc&buy_now=0&urgent=0&page={p}'
    print(bikroy_url)


    r = session.get('http://localhost:8050/render.html', params={'url': bikroy_url, 'wait': 1})
    print(r.status_code)

#     driver.get(bikroy_url)
    time.sleep(3)
    page_cnt += 1
    for i in range (1, 28):
        try:
#             single_url = driver.find_element(by=By.XPATH, value=f'//*[@id="app-wrapper"]/div[1]/div[3]/div/div[2]/div[4]/div[2]/div/div[1]/div[1]/ul/li[{i}]/a').get_attribute('href')
            single_url = r.html.find(f'#app-wrapper > div.container--297Nx.all > div:nth-child(3) > div > div.row--3Vhjr.main-container--28aG5.justify-content-flex-start--1Xozy.align-items-normal--vaTgD.flex-wrap-wrap--2PCx8.flex-direction-row--27fh1.flex--3fKk1 > div.sm-col-12--30zDS.lg-col-9--20qCf.block--3v-Ow > div.ad-list-container--1UnyA > div > div.ad-list--2Y3ql > div.list-wrapper--t_A02 > ul > li:nth-child({i}) > a')
            single_url = single_url[0].attrs['href']
            single_page_urls.append(single_url)
            url_cnt += 1
            print("Page: " + str(page_cnt) + " URL: " + str(url_cnt))
        except:
            print('Not details link')
    
    clear_output(wait=True)
print(time.time()-start)
# driver.maximize_window()

524.0431141853333


In [5]:
len(single_page_urls), single_page_urls

(2673,
 ['/en/ad/suzuki-solio-g-edition-push-start-2011-for-sale-dhaka',
  '/en/ad/toyota-noah-x-sun-roof-octane-2004-for-sale-dhaka',
  '/en/ad/baaccaader-dui-tti-saaikel-bikriyy-kraa-hbe-for-sale-dhaka',
  '/en/ad/tvs-metro-kono-kaaj-naai-2013-for-sale-khulna-division',
  '/en/ad/ttaakaa-pryyojn-taai-sel-dib-for-sale-mymensingh',
  '/en/ad/dayang-dy-150-dayun-2013-for-sale-khulna',
  '/en/ad/tvs-apache-rtr-4v-dd-sohaa-mttrs-2020-for-sale-dhaka-division-4',
  '/en/ad/mitsubishi-outlander-octane-draive-2008-for-sale-dhaka-1',
  '/en/ad/hero-hunk-double-disk-2018-for-sale-chattogram-division',
  '/en/ad/rcb-s1-master-brake-pump-for-sale-dhaka',
  '/en/ad/tvs-apache-rtr-2019-for-sale-dhaka-division-304',
  '/en/ad/beetle-bolt-alligator-good-2021-for-sale-dhaka-division',
  '/en/ad/wuyung-ad-80s-2014-for-sale-khulna-division',
  '/en/ad/toyota-dyna-pick-up-dhaka-ready-2016-for-sale-dhaka',
  '/en/ad/toyota-corolla-111-1998-for-sale-chattogram-65',
  '/en/ad/walton-fusion-2017-2015-for-sal

In [21]:
temp = single_url[0].attrs['href']
temp

'/en/ad/dell-latitude-7370-touchintel-core-m7-6y57-8gb-ddr3-ram-256gb-ssd-for-sale-dhaka'

In [4]:
with open('home-living_product_urls.txt', 'w') as f:
    for s in single_page_urls:
        f.write(s)
        f.write('\n')

In [31]:
def title_scrap(driver):
    title = driver.find_element(by=By.TAG_NAME, value='h1').text
    print("Title: ", s_title)
    #     //*[@id="app-wrapper"]/div[1]/div[2]/div[2]/div[1]/div[1]/div/div[1]/div/div[1]/div[2]/div/div/h1
    #     //*[@id="app-wrapper"]/div[1]/div[2]/div[2]/div[1]/div[1]/div/div[1]/div/div[1]/div[2]/div/div/h1

    return title

In [35]:
def time_location_scrap(driver):
    time_location = driver.find_element(by=By.CLASS_NAME, value='sub-title--37mkY').text
    print("Time & Location: ", time_location)

    return time_location

In [36]:
def price_scrap(driver):
    price = driver.find_element(by=By.CLASS_NAME, value='amount--3NTpl').text
    print("Price: ", price)

    return price

In [16]:
def image_scrap(driver):
    try:
        images = driver.find_element(by=By.XPATH, value='//*[@id="app-wrapper"]/div[1]/div[2]/div[2]/div[3]/div[2]/div/div[1]/div/div[1]/div/div/ul')
        img_path = []
        for img in images.find_elements(by=By.TAG_NAME,value='img'):
            img_urls = img.get_attribute('src')

            img_urls_firstname = '/'.join(img_urls.split('/')[:-3])
            img_urls_rename = img_urls_firstname + '/620/466/fitted.jpg'
    #         print(img_urls_rename)
            img_name = '@'.join(img_urls_firstname.split('/')[-2:])

            r = requests.get(img_urls_rename)
            i = Image.open(BytesIO(r.content))
            i.save(f"img/mobile/{img_name}.jpg")
            img_path.append(img_name)
        return img_path
    except:
        print('No images')        
        return 'N/A'

In [8]:
def meta_data_scrap(driver):
    try:
        meta_data = driver.find_element(by=By.XPATH, value='//*[@id="app-wrapper"]/div[1]/div[2]/div[2]/div[3]/div[2]/div/div[1]/div/div[2]/div[2]')

        md = meta_data.find_elements(by=By.TAG_NAME,value='div')
        meta_dic = {}
        for m in md:
            m = m.text
            if '\n' in m:
                key, val = m.split('\n')[0].replace(':', ''), m.split('\n')[1]
                meta_dic[key] = val
        print("Meta Data: ", meta_dic)
        return meta_dic
    except:
        print('No meta data found')        
        return 'N/A'

In [9]:
def show_more_click(driver):    
    try:
        s_driver.find_element(by=By.CLASS_NAME, value='show-text--13FcL').click()
#         s_driver.find_element(by=By.XPATH, value='//*[@id="app-wrapper"]/div[1]/div[2]/div[2]/div[3]/div[2]/div/div[1]/div/div[2]/div[3]/div/div[2]/div/div[2]/button').click()
        time.sleep(2)
    except:
        print("No 'Show more' button")

In [10]:
def feature_scrap(driver):
    try:
        feature = driver.find_element(by=By.CLASS_NAME, value='text--Xap4O').text
#         feature = s_driver.find_element(by=By.XPATH, value='//*[@id="app-wrapper"]/div[1]/div[2]/div[2]/div[1]/div[1]/div/div[1]/div/div[2]/div[4]/p').text
#         feature = feature.find_elements(by=By.TAG_NAME,value='p').text
        print('Features: ', feature)
        return feature
    except:
        print("No feature")        
        return 'N/A'

In [18]:
def description_scrap(driver):
    # desc = s_driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div[2]/div[2]/div[3]/div[2]/div/div[1]/div/div[2]/div[3]/div/div[2]/div/div[1]/div/ul/div')
    # description[0].text
    # description = [d.text.replace("'\uf076", "").replace("''\uf0d8", "") for d in desc.find_elements_by_tag_name('p')]
    # print(description)

    try:
        desc = s_driver.find_element(by=By.XPATH, value='//*[@id="collapsible-content-0"]/ul/div').text
        print("Description: ", desc)
        return desc
    except:
        print("No description")        
        return 'N/A'

In [42]:
op = webdriver.ChromeOptions()
op.add_argument('headless')
s = Service('chromedriver')
s_driver = webdriver.Chrome(service=s, options=op)

cnt = 1
titles, locations, prices, img_lst, meta_lst,  feature_lst, desc_lst = [], [], [], [], [], [], []
for s_u in single_page_urls[:1]:
    print("Count: ", cnt)
    s_driver.get(s_u)
    time.sleep(3)

    # single page data scrapping
    s_title = title_scrap(s_driver)
    titles.append(s_title)
    
    # scrapping time and location
    time_location = time_location_scrap(s_driver)
    locations.append(time_location)
    
    # scrapping price
    price = price_scrap(s_driver)
    prices.append(price)
    
    # scrapping all images
    images = image_scrap(s_driver)
    img_lst.append(images)
    

    # extractig meta data
    meta_data = meta_data_scrap(s_driver)
    meta_lst.append(meta_data)
        
    # click on show_more button
    show_more_click(s_driver)

    # extracting feature data
    feature = feature_scrap(s_driver)
    feature_lst.append(feature)
    
    # extracting description
    description = description_scrap(s_driver)
    desc_lst.append(description)
    
    cnt += 1
    print("=============================================")
    clear_output(wait=True)

Count:  1
Title:  OnePlus 9 Pro 12GB 256GB global (New)
Time & Location:  Posted on 06 Mar 6:25 pm, CDA Avenue, Chattogram
Price:  Tk 63,990
Meta Data:  {'Condition': 'New', 'Brand': 'OnePlus', 'Model': '9 Pro', 'Edition': '12GB 256GB global', 'Authenticity': 'Original'}
No feature
Description:  @@  new and pack❤️😍
14 days Replacement Guarantee and 1 year warranty. 
🎯 Exchange possible_Best offer. 
♻️Buy, Sell & Exchange♻️
💧Full Box with all Original Accessories.
📷 Original Picture Uploaded 📷
🔥We sell 100% Original & Non Refurbished Product.
📌 আপনার পুরনো স্মার্টফোনটি দিয়ে #EXCHANGE করে নতুন ফোন কিনতে পারবেন। 
এছাড়াও আমরা পুরনো ফোন ক্রয়ও করে থাকি
14দিনের রিপ্লেস্মেন্ট ওয়ারেন্টি। ( MI তে ২৪ মাস সার্ভিস ওয়ারেন্টি)
আপনি আমাদের থেকে পাচ্ছেন ঃSAMSUNG,XIAOMI, Realme, NOKIA, vivo, oppo,, Apple , Motorola, মোবাইল
আমাদের সুবিধা সমূহ ঃ
১)অরিজিনাল প্রোডাক্ট এর গ্যারান্টি
২)১৪ দিন এর রিপ্লেসমেন্ট সুবিধা
৩)এক্সচেজ্ এর সুবিধা 
৪) ৩- ৩৬মাস এর কিস্তি সুবিধা 
আপনাদের জন্য মারবেল দিচ্ছে ৩,৬,৯,১২,১৮,২৪,৩

In [44]:
titles, meta_lst

(['OnePlus 9 Pro 12GB 256GB global (New)'],
 [{'Condition': 'New',
   'Brand': 'OnePlus',
   'Model': '9 Pro',
   'Edition': '12GB 256GB global',
   'Authenticity': 'Original'}])

In [ ]:
df_lst = [titles, img_lst, meta_lst,  feature_lst, desc_lst]
column_lst = ['Title', 'Images', 'Meta Data', 'Feature', 'Description']
df = pd.DataFrame (df_lst, columns = column_lst)
df.to_csv('Bikroy-Mobile-Category.csv', index=False)